In [3]:
import pandas as pd
# !pip install transitionMatrix
import os  

In [9]:

for j in range(220,320):
    df_checkinpd=pd.read_csv(r'C:\Users\dell\Documents\bia678\markov chain\client1\client'+str(j)+'.csv')
    len_whole=len(df_checkinpd)
    df_checkinpd[['date','time']] = df_checkinpd.timestamp.str.split(" ",expand=True)
    df_checkinpd[['time_1','time_2','time_3']] = df_checkinpd.time.str.split(":",expand=True)
    df_checkinpd.drop(["time_3"], axis=1, inplace=True)
    def warn(*args, **kwargs):
        pass
        import warnings
        warnings.warn = warn
    list_date=list(df_checkinpd.date.unique())
    dfArray = [df_checkinpd.loc[df_checkinpd.date == x] for x in list_date]
    def get_set(df_1):
        set1=[df_1.index[0]]
        index_end=0
        for index, row in df_1.iterrows():
            index_start=index
            for index_2 in range(index_start,df_1.index[-1]+1):
                if (row['building']!=df_1['building'][index_2]):
                    index_end=index_2
                    set1.append(index_2)
                    break
        set1.append(df_1.index[-1])
        return set1
    def get_list(df_1,list1):
        len2=len(list1)
        list2=[]
        time_period=0
        for i in range(len2-1):
            if list1[i+1]-list1[i]==1: time_period=0;
            else: time_period=(int(df_1['time_1'][list1[i+1]-1])*60+int(df_1['time_2'][list1[i+1]-1]))-(int(df_1['time_1'][list1[i]])*60+int(df_1['time_2'][list1[i]]))
            list2.append(time_period)
        return list2
    list_1,list_2,list_3,list_4=[],[],[],[]
    for i in range(len(list_date)):
        list_1=list(set(get_set(dfArray[i])))
        list_1.sort()
        list_4=list_4+list_1[:len(list_1)-1]
        list_2=get_list(dfArray[i],list_1)
        list_3=list_3+list_2
    df_state=df_checkinpd.loc[list_4,:]
    df_state['stay time']=list_3
    df_state.drop(['client_df_2','AP','idx','client1'],axis=1,inplace=True) 
    df_state_1=df_state.loc[df_state['stay time']>=120] 
    pd.set_option('display.max_rows', 20) 
    # df_state_1['date1']=pd.to_datetime(df_state_1['date'],infer_datetime_format=True)
    df_state_1['date_1']= pd.to_datetime(df_state_1['date']).dt.dayofweek
    df_state_1['date_day of week']=df_state_1['date_1'].map({0: 'Monday', 1: 'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})
    len_state1=len(df_state_1)
    s=list(df_state_1['time_1'])
    len_first=len(s)
    list2=[]
    for list1 in s:
        if int(list1)<12: list2.append('Morning')
        if (int(list1)>=12) & (int(list1)<19): list2.append('afternoon')
        if int(list1)>=19: list2.append('Evening')
    len_last=len(list2)
    if len_first==len_last: print("correct conversion")
    df_state_1['each_day']=list2
    df_state_1['observation']=df_state_1['date_day of week']+'_'+df_state_1.each_day
    df_markov=df_state_1
    df_markov=df_markov[['building','stay time','observation']]
    list_ob=df_markov.observation.unique()
    list_building=df_markov.building.unique()
    list_building = list_building[~pd.isnull(list_building)]
    list_ob = list_ob[~pd.isnull(list_ob)]
    df_markov=df_markov.reset_index(drop=True)
    df_state_1=df_state_1.reset_index(drop=True)
    df_check=df_state_1
    df_markov=df_state_1
    list_datafrme=[]
    def even(df_markov,building):
        list_time=[]
        list_time_1=[]
        df_label=df_markov[df_markov['building']==building]
        sum1=df_label['stay time'].sum()
        list_time=list(df_label['stay time'])
        for i in list_time:
            list_time_1.append(round(i/sum1,2))
        df_label['prob']=list_time_1
        return df_label
    for building in list_building: 
        df_even=even(df_markov,building)
        list_datafrme.append(df_even)
    df_markov_1=pd.concat(list_datafrme)
    df_markov_1['combine']=df_markov_1.observation+'/'+df_markov_1.building
    list_combine=df_markov_1['combine'].unique()
    result={}
    def even(df_markov,combine):
        dict1={}
        df_label=df_markov[df_markov['combine']==combine]
        sum1=df_label['prob'].sum()
        dict1[combine]=sum1
        return dict1
    for combine in list_combine: 
        result.update(even(df_markov_1,combine))
    df_markov_2=pd.DataFrame(list(result.items()), columns=['combine', 'prob'])
    df_markov_2[['observation','hidden']] = df_markov_2['combine'].str.split("/",expand=True)
    df_markov_2=df_markov_2.drop(columns='combine')
    markov_emission=pd.DataFrame(columns=list_ob,index=list_building)
#     if (len(df_markov_2)<35) or
    if (len(df_markov_2['observation'].unique())<3): continue
    for index,row in markov_emission.iterrows():
        for index1,row1 in df_markov_2.iterrows():
            markov_emission[row1['observation']][row1['hidden']]=row1['prob']
    markov_emission.to_csv(r'C:\Users\dell\Documents\bia678\markov chain\a1\markov_emission_client'+str(j)+'.csv')
    df_markov.to_csv(r'C:\Users\dell\Documents\bia678\markov chain\a2\df_markov_client'+str(j)+'.csv')
    df_markov_2.to_csv(r'C:\Users\dell\Documents\bia678\markov chain\a3\df_markov_2_client'+str(j)+'.csv')

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conversion
correct conv